In [ ]:
"""
Prepare cav mean and cav std

"""

In [ ]:
import xarray as xr
import multimelt.useful_functions as uf
from tqdm.notebook import tqdm

READ IN DATA

In [ ]:
nemo_run = 'bf663' #'bi646'

In [ ]:
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'


In [ ]:
for tt in tqdm(range(1970, 1970 + 72)): #continue at 1978

    # T and S extrapolated to ice draft depth
    T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc').squeeze().drop('time')

    # dIF, dGL
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc').drop('time')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')
    
    n = 0
    for kisf in file_isf.Nisf:
        #conc_isf = uf.choose_isf(file_conc_cut['isfdraft_conc'],isf_stack_mask, kisf)
        T_isf = uf.choose_isf(T_S_2D_isfdraft['theta_in'],isf_stack_mask, kisf)
        S_isf = uf.choose_isf(T_S_2D_isfdraft['salinity_in'],isf_stack_mask, kisf)

        T_mean_cav = T_isf.mean('mask_coord').to_dataset(name='T_mean')
        S_mean_cav = S_isf.mean('mask_coord').to_dataset(name='S_mean')
        T_std_cav = T_isf.std('mask_coord').to_dataset(name='T_std')
        S_std_cav = S_isf.std('mask_coord').to_dataset(name='S_std')

        all_TS_isf = xr.merge([T_mean_cav,S_mean_cav,T_std_cav,S_std_cav])
        all_TS_isf_2D = all_TS_isf.where(T_isf).load()

        if n == 0:
            all_TS_all = all_TS_isf_2D.squeeze().drop('Nisf')
        else:
             all_TS_all =  all_TS_all.combine_first(all_TS_isf_2D).squeeze().drop('Nisf')
        n = n+1

        del all_TS_isf_2D

    all_TS_all_unstacked = uf.bring_back_to_2D(all_TS_all)
    
    all_TS_all_unstacked.to_netcdf(inputpath_profiles + 'T_S_2D_meanstd_isf_draft_oneFRIS_'+str(tt)+'.nc')